In [1]:
import os
import re
import pandas as pd
import numpy as np

In [2]:
def change_dtype_ser(ser):
    
    if ser.dtype == int:
        return ser.astype(np.int32)
    
    if ser.dtype == float:
        return ser.astype(np.float32)
    
    if ser.dtype == np.object:
        return ser.astype("category")
    
    return ser
    

def change_dtype_df(df):
    """
    change types of columns to reduce memory size
    :param df: dataframe
    :return df: dataframe
    """
    df["fecha_dato"] = pd.to_datetime(df["fecha_dato"])
    df["fecha_alta"] = pd.to_datetime(df["fecha_alta"])
    
    memory = df.memory_usage().sum() / 10**6
    print("Memory usage before changing types %0.2f MB" % memory)

    for col in df.columns:
        df[col] = change_dtype_ser(df[col])

    memory = df.memory_usage().sum() / 10 ** 6
    print("Memory usage after changing types %0.2f MB" % memory)
    return df


def load_csv(filename):
    df = pd.read_csv(filename)
    df = change_dtype_df(df)
    return df

In [3]:
INP_DIR = "data/data_"
OUT_DIR1 = "data/data1_"
OUT_DIR2 = "data/data2_"
OUT_DIR3 = "data/data3_"

In [4]:
df_train = load_csv(os.path.join(INP_DIR, "train_cleaned.csv"))
df_test = load_csv(os.path.join(INP_DIR, "test_cleaned.csv"))

/home/hai/opt/python_virtual_environments/xgboost/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Memory usage before changing types 9566.76 MB
Memory usage after changing types 4612.80 MB
Memory usage before changing types 116.20 MB
Memory usage after changing types 49.28 MB


In [5]:
# drop this column becuase it is too imbalanced
df_train = df_train.drop(["ind_empleado"], axis=1)
df_test = df_test.drop(["ind_empleado"], axis=1)

In [6]:
# days from when the data is recorded 
df_train["fecha_alta"] = (df_train["fecha_alta"] - df_train["fecha_dato"]).dt.days
df_test["fecha_alta"] = (df_test["fecha_alta"] - df_test["fecha_dato"]).dt.days

In [7]:
df_train.isnull().sum().sum(), df_test.isnull().sum().sum()

(0, 0)

In [8]:
df_train.shape, df_test.shape

((13647309, 91), (929615, 19))

In [9]:
df_train.columns

Index(['fecha_dato', 'ncodpers', 'pais_residencia', 'sexo', 'age',
       'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes',
       'tiprel_1mes', 'indresi', 'indext', 'canal_entrada', 'indfall',
       'cod_prov', 'ind_actividad_cliente', 'renta', 'segmento',
       'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1',
       'ind_ahor_fin_ult1_NEW_PUR', 'ind_aval_fin_ult1_NEW_PUR',
       'ind_cco_fin_ult1_NEW_PUR', 'ind_cder_fin_ult1_NEW_PUR',
       'ind_cno_fin_ult1_NEW_PUR', 'ind_ctj

In [10]:
df_test.columns

Index(['fecha_dato', 'ncodpers', 'pais_residencia', 'sexo', 'age',
       'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes',
       'tiprel_1mes', 'indresi', 'indext', 'canal_entrada', 'indfall',
       'cod_prov', 'ind_actividad_cliente', 'renta', 'segmento'],
      dtype='object')

In [11]:
PROD_COLS = [col for col in df_train.columns if re.match(r"^ind_.*_ult1$", col)]
print(PROD_COLS)

['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']


In [12]:
# total number of prod
df_train["TOTAL_PRODS"] = df_train[PROD_COLS].sum(axis=1)

In [13]:
NEW_PUR_COLS = [col for col in df_train.columns if re.match(r"^ind_.*_ult1_NEW_PUR$", col)]
print(NEW_PUR_COLS)

['ind_ahor_fin_ult1_NEW_PUR', 'ind_aval_fin_ult1_NEW_PUR', 'ind_cco_fin_ult1_NEW_PUR', 'ind_cder_fin_ult1_NEW_PUR', 'ind_cno_fin_ult1_NEW_PUR', 'ind_ctju_fin_ult1_NEW_PUR', 'ind_ctma_fin_ult1_NEW_PUR', 'ind_ctop_fin_ult1_NEW_PUR', 'ind_ctpp_fin_ult1_NEW_PUR', 'ind_deco_fin_ult1_NEW_PUR', 'ind_deme_fin_ult1_NEW_PUR', 'ind_dela_fin_ult1_NEW_PUR', 'ind_ecue_fin_ult1_NEW_PUR', 'ind_fond_fin_ult1_NEW_PUR', 'ind_hip_fin_ult1_NEW_PUR', 'ind_plan_fin_ult1_NEW_PUR', 'ind_pres_fin_ult1_NEW_PUR', 'ind_reca_fin_ult1_NEW_PUR', 'ind_tjcr_fin_ult1_NEW_PUR', 'ind_valo_fin_ult1_NEW_PUR', 'ind_viv_fin_ult1_NEW_PUR', 'ind_nomina_ult1_NEW_PUR', 'ind_nom_pens_ult1_NEW_PUR', 'ind_recibo_ult1_NEW_PUR']


In [14]:
PUR_CANCEL_COLS = [col for col in df_train.columns if re.match(r"^ind_.*_ult1_PUR_OR_CANCEL$", col)]
print(PUR_CANCEL_COLS)

['ind_ahor_fin_ult1_PUR_OR_CANCEL', 'ind_aval_fin_ult1_PUR_OR_CANCEL', 'ind_cco_fin_ult1_PUR_OR_CANCEL', 'ind_cder_fin_ult1_PUR_OR_CANCEL', 'ind_cno_fin_ult1_PUR_OR_CANCEL', 'ind_ctju_fin_ult1_PUR_OR_CANCEL', 'ind_ctma_fin_ult1_PUR_OR_CANCEL', 'ind_ctop_fin_ult1_PUR_OR_CANCEL', 'ind_ctpp_fin_ult1_PUR_OR_CANCEL', 'ind_deco_fin_ult1_PUR_OR_CANCEL', 'ind_deme_fin_ult1_PUR_OR_CANCEL', 'ind_dela_fin_ult1_PUR_OR_CANCEL', 'ind_ecue_fin_ult1_PUR_OR_CANCEL', 'ind_fond_fin_ult1_PUR_OR_CANCEL', 'ind_hip_fin_ult1_PUR_OR_CANCEL', 'ind_plan_fin_ult1_PUR_OR_CANCEL', 'ind_pres_fin_ult1_PUR_OR_CANCEL', 'ind_reca_fin_ult1_PUR_OR_CANCEL', 'ind_tjcr_fin_ult1_PUR_OR_CANCEL', 'ind_valo_fin_ult1_PUR_OR_CANCEL', 'ind_viv_fin_ult1_PUR_OR_CANCEL', 'ind_nomina_ult1_PUR_OR_CANCEL', 'ind_nom_pens_ult1_PUR_OR_CANCEL', 'ind_recibo_ult1_PUR_OR_CANCEL']


In [15]:
DEMOG_COLS = [col for col in df_train.columns 
    if col not in PROD_COLS + NEW_PUR_COLS + PUR_CANCEL_COLS + ["fecha_dato", "ncodpers", "TOTAL_PRODS"]]
print(DEMOG_COLS)

['pais_residencia', 'sexo', 'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext', 'canal_entrada', 'indfall', 'cod_prov', 'ind_actividad_cliente', 'renta', 'segmento']


In [16]:
DEMOG_LAG_COLS = ["ind_nuevo", "antiguedad", "indrel", "tiprel_1mes", 
            "ind_actividad_cliente", "renta", "segmento", "TOTAL_PRODS"]

LAG_COLS = PROD_COLS + PUR_CANCEL_COLS + DEMOG_LAG_COLS
LAG_COLS

['ind_ahor_fin_ult1',
 'ind_aval_fin_ult1',
 'ind_cco_fin_ult1',
 'ind_cder_fin_ult1',
 'ind_cno_fin_ult1',
 'ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1',
 'ind_ctop_fin_ult1',
 'ind_ctpp_fin_ult1',
 'ind_deco_fin_ult1',
 'ind_deme_fin_ult1',
 'ind_dela_fin_ult1',
 'ind_ecue_fin_ult1',
 'ind_fond_fin_ult1',
 'ind_hip_fin_ult1',
 'ind_plan_fin_ult1',
 'ind_pres_fin_ult1',
 'ind_reca_fin_ult1',
 'ind_tjcr_fin_ult1',
 'ind_valo_fin_ult1',
 'ind_viv_fin_ult1',
 'ind_nomina_ult1',
 'ind_nom_pens_ult1',
 'ind_recibo_ult1',
 'ind_ahor_fin_ult1_PUR_OR_CANCEL',
 'ind_aval_fin_ult1_PUR_OR_CANCEL',
 'ind_cco_fin_ult1_PUR_OR_CANCEL',
 'ind_cder_fin_ult1_PUR_OR_CANCEL',
 'ind_cno_fin_ult1_PUR_OR_CANCEL',
 'ind_ctju_fin_ult1_PUR_OR_CANCEL',
 'ind_ctma_fin_ult1_PUR_OR_CANCEL',
 'ind_ctop_fin_ult1_PUR_OR_CANCEL',
 'ind_ctpp_fin_ult1_PUR_OR_CANCEL',
 'ind_deco_fin_ult1_PUR_OR_CANCEL',
 'ind_deme_fin_ult1_PUR_OR_CANCEL',
 'ind_dela_fin_ult1_PUR_OR_CANCEL',
 'ind_ecue_fin_ult1_PUR_OR_CANCEL',
 'ind_fond_fin

In [17]:
def extract_subset(df, row_filter, cols):
    return df.loc[row_filter, cols]



def suffixing_cols(df, suffix="_LAG", exclude_cols=["ncodpers"]):
    new_cols = [col + suffix if col not in exclude_cols else col for col in df.columns]
    df.columns = new_cols 
    return df



def extract_by_timestamp_newpur(df, timestamp, sel_prod_cols, 
                                new_pur_suffix="_NEW_PUR", 
                                sel_cols=DEMOG_COLS):
    """extract rows by timestamp and new purchase columns"""
    df = df.copy()
    
    if "ncodpers" not in sel_cols:
        sel_cols = ["ncodpers"] + sel_cols
    
    extracted_dfs = []
    targets = []
    for prod_col in sel_prod_cols:
        pur_col = prod_col + new_pur_suffix
        row_filter = (df["fecha_dato"] == timestamp) & (df[pur_col] == 1)
        
        sub_df = extract_subset(df, row_filter, sel_cols)
        #print("sub_df.shape:", sub_df.shape)
        extracted_dfs.append(sub_df)
        
        ys = [prod_col] * sub_df.shape[0]
        targets.extend(ys)
    
    extracted_dfs = pd.concat(extracted_dfs, axis=0).reset_index(drop=True)
    extracted_dfs["TARGET"] = targets
    
    extracted_dfs = extracted_dfs.sort_values(by="ncodpers")
    return extracted_dfs


def extract_by_timestamp_custid(df, timestamp, customer_ids, sel_cols):
    df = df.copy()
    
    customer_ids = np.unique(customer_ids)
    
    if "ncodpers" not in sel_cols:
        sel_cols = ["ncodpers"] + sel_cols
    #print("sel_cols:", sel_cols)
    
    row_filter = (df["fecha_dato"] == timestamp) & df["ncodpers"].isin(customer_ids)
    df_out = extract_subset(df, row_filter, sel_cols)
    return df_out


def extract_X_y_train(df, timestamp, timestamp_lags, sel_prod_cols,
                      demog_cols=DEMOG_COLS, lag_cols=LAG_COLS):
    out_df = extract_by_timestamp_newpur(df, timestamp, sel_prod_cols, sel_cols=demog_cols)
    
    customer_ids = out_df["ncodpers"]
    for t, lag in enumerate(timestamp_lags):
        assert pd.to_datetime(lag) < pd.to_datetime(timestamp), lag + " lag is not before timestamp " +  timestamp
        lag_label = "_LAG%d" % (t + 1)
        
        lag_df = extract_by_timestamp_custid(df, lag, customer_ids=customer_ids, sel_cols=lag_cols)
        lag_df = suffixing_cols(lag_df, lag_label, exclude_cols=["ncodpers"])
        
        print(lag, lag_label, lag_df.shape)
        
        out_df = out_df.merge(lag_df, how="left", on="ncodpers")
    return out_df


def extract_X_test(train, test, timestamp, timestamp_lags, demog_cols=DEMOG_COLS, lag_cols=LAG_COLS):
    customer_ids = test["ncodpers"]
    out_df = extract_by_timestamp_custid(test, timestamp, customer_ids=customer_ids, sel_cols=demog_cols)
    
    for t, lag in enumerate(timestamp_lags):
        assert pd.to_datetime(lag) < pd.to_datetime(timestamp), lag + " lag is not before timestamp " +  timestamp
        lag_label = "_LAG%d" % (t + 1)
        
        lag_df = extract_by_timestamp_custid(train, lag, customer_ids=customer_ids, sel_cols=lag_cols)
        lag_df = suffixing_cols(lag_df, lag_label, exclude_cols=["ncodpers"])
        
        print(lag, lag_label, lag_df.shape)
        
        out_df = out_df.merge(lag_df, how="left", on="ncodpers")
    return out_df

In [18]:
def most_popular_purchase(df, timestamp, pur_cols, suffix="_NEW_PUR", threshold=None):
    prod_popul = df.loc[df["fecha_dato"] == timestamp, pur_cols].sum(axis=0)
    prod_popul = prod_popul.sort_values(ascending=False)
    prod_popul.index = [idx.replace(suffix, "") for idx in prod_popul.index]
    prod_popul = prod_popul[prod_popul > 0]
    
    if threshold is None:
        index = prod_popul.index
    else:
        index = prod_popul.index[:threshold]
    return prod_popul[index]

In [19]:
df_train["fecha_dato"].unique()

array(['2015-01-28T00:00:00.000000000', '2015-02-28T00:00:00.000000000',
       '2015-03-28T00:00:00.000000000', '2015-04-28T00:00:00.000000000',
       '2015-05-28T00:00:00.000000000', '2015-06-28T00:00:00.000000000',
       '2015-07-28T00:00:00.000000000', '2015-08-28T00:00:00.000000000',
       '2015-09-28T00:00:00.000000000', '2015-10-28T00:00:00.000000000',
       '2015-11-28T00:00:00.000000000', '2015-12-28T00:00:00.000000000',
       '2016-01-28T00:00:00.000000000', '2016-02-28T00:00:00.000000000',
       '2016-03-28T00:00:00.000000000', '2016-04-28T00:00:00.000000000',
       '2016-05-28T00:00:00.000000000'], dtype='datetime64[ns]')

# Use `2016-05`  with 6-month lags to predict `2016-06`

In [21]:
popul_pur_2016_04 = most_popular_purchase(df_train, "2016-04-28", NEW_PUR_COLS)
popul_pur_2016_05 = most_popular_purchase(df_train, "2016-05-28", NEW_PUR_COLS, threshold=16)

In [22]:
popul_pur_2016_05

ind_recibo_ult1      10163
ind_nom_pens_ult1     5513
ind_nomina_ult1       5488
ind_tjcr_fin_ult1     4248
ind_cco_fin_ult1      3878
ind_ecue_fin_ult1     2709
ind_cno_fin_ult1      2347
ind_ctma_fin_ult1      531
ind_reca_fin_ult1      279
ind_ctop_fin_ult1      226
ind_valo_fin_ult1      183
ind_ctpp_fin_ult1      131
ind_fond_fin_ult1       61
ind_dela_fin_ult1       46
ind_ctju_fin_ult1       40
ind_plan_fin_ult1       22
dtype: int64

In [23]:
Y_LABES = set(popul_pur_2016_04.index).intersection(set(popul_pur_2016_05.index))
Y_LABES = list(Y_LABES)
print(Y_LABES)
len(Y_LABES)

['ind_ctju_fin_ult1', 'ind_ctop_fin_ult1', 'ind_dela_fin_ult1', 'ind_cno_fin_ult1', 'ind_nom_pens_ult1', 'ind_reca_fin_ult1', 'ind_valo_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_fond_fin_ult1', 'ind_nomina_ult1', 'ind_recibo_ult1', 'ind_ecue_fin_ult1', 'ind_plan_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_ctma_fin_ult1', 'ind_cco_fin_ult1']


16

## Extract `X_2016_04`, `y_2016_04`

In [35]:
timestamp = "2016-04-28"
timestamp_lags = ["2016-03-28", "2016-02-28", "2016-01-28", 
                  "2015-12-28", "2015-11-28", "2015-10-28"]

X_y_2016_04 = extract_X_y_train(df_train, timestamp, timestamp_lags, Y_LABES)
X_y_2016_04.shape

2016-03-28 _LAG1 (26728, 57)
2016-02-28 _LAG2 (25043, 57)
2016-01-28 _LAG3 (24247, 57)
2015-12-28 _LAG4 (23548, 57)
2015-11-28 _LAG5 (22797, 57)
2015-10-28 _LAG6 (22300, 57)


(33005, 355)

In [36]:
X_y_2016_04.to_csv(os.path.join(OUT_DIR1, "X_y_2016_04.csv"), index=False)

## Extract `X_2016_05`, `y_2016_05`

In [37]:
timestamp = "2016-05-28"
timestamp_lags = ["2016-04-28", "2016-03-28", "2016-02-28", "2016-01-28",
                  "2015-12-28", "2015-11-28"]

X_y_2016_05 = extract_X_y_train(df_train, timestamp, timestamp_lags, Y_LABES)
X_y_2016_05.shape

2016-04-28 _LAG1 (27856, 57)
2016-03-28 _LAG2 (26353, 57)
2016-02-28 _LAG3 (25674, 57)
2016-01-28 _LAG4 (24918, 57)
2015-12-28 _LAG5 (23950, 57)
2015-11-28 _LAG6 (23535, 57)


(35865, 355)

In [38]:
X_y_2016_05.to_csv(os.path.join(OUT_DIR1, "X_y_2016_05.csv"), index=False)

## Extract `X_2016_06`

In [39]:
timestamp = "2016-06-28"
timestamp_lags = ["2016-05-28", "2016-04-28", "2016-03-28", "2016-02-28", "2016-01-28",
                  "2015-12-28"]

X_2016_06 = extract_X_test(df_train, df_test, timestamp, timestamp_lags)
X_2016_06.shape

2016-05-28 _LAG1 (929615, 57)
2016-04-28 _LAG2 (925252, 57)
2016-03-28 _LAG3 (920975, 57)
2016-02-28 _LAG4 (915679, 57)
2016-01-28 _LAG5 (909885, 57)
2015-12-28 _LAG6 (903429, 57)


(929615, 354)

In [40]:
X_2016_06.to_csv(os.path.join(OUT_DIR1, "X_2016_06.csv"), index=False)

# Use `2016-05`  with 12-month lags to predict `2016-06`

## Extract `X_2016_04`, `y_2016_04`

In [41]:
timestamp = "2016-04-28"
timestamp_lags = ["2016-03-28", "2016-02-28", "2016-01-28", 
                  "2015-12-28", "2015-11-28", "2015-10-28",
                  "2015-09-28", "2015-08-28", "2015-07-28", 
                  "2015-06-28", "2015-05-28", "2015-04-28"]

X_y_2016_04 = extract_X_y_train(df_train, timestamp, timestamp_lags, Y_LABES)
X_y_2016_04.shape

2016-03-28 _LAG1 (26728, 57)
2016-02-28 _LAG2 (25043, 57)
2016-01-28 _LAG3 (24247, 57)
2015-12-28 _LAG4 (23548, 57)
2015-11-28 _LAG5 (22797, 57)
2015-10-28 _LAG6 (22300, 57)
2015-09-28 _LAG7 (21811, 57)
2015-08-28 _LAG8 (21407, 57)
2015-07-28 _LAG9 (21135, 57)
2015-06-28 _LAG10 (20215, 57)
2015-05-28 _LAG11 (20056, 57)
2015-04-28 _LAG12 (19878, 57)


In [42]:
X_y_2016_04.to_csv(os.path.join(OUT_DIR2, "X_y_2016_04.csv"), index=False)

## Extract `X_2016_05`, `y_2016_05`

In [45]:
timestamp = "2016-05-28"
timestamp_lags = ["2016-04-28", "2016-03-28", "2016-02-28", "2016-01-28",
                  "2015-12-28", "2015-11-28", "2015-10-28", "2015-09-28",
                 "2015-08-28", "2015-07-28", "2015-06-28", "2015-05-28"]

X_y_2016_05 = extract_X_y_train(df_train, timestamp, timestamp_lags, Y_LABES)
X_y_2016_05.shape

2016-04-28 _LAG1 (27856, 57)
2016-03-28 _LAG2 (26353, 57)
2016-02-28 _LAG3 (25674, 57)
2016-01-28 _LAG4 (24918, 57)
2015-12-28 _LAG5 (23950, 57)
2015-11-28 _LAG6 (23535, 57)
2015-10-28 _LAG7 (23161, 57)
2015-09-28 _LAG8 (22707, 57)
2015-08-28 _LAG9 (22297, 57)
2015-07-28 _LAG10 (22036, 57)
2015-06-28 _LAG11 (21093, 57)
2015-05-28 _LAG12 (20946, 57)


(35865, 691)

In [46]:
X_y_2016_05.to_csv(os.path.join(OUT_DIR2, "X_y_2016_05.csv"), index=False)

## Extract `X_2016_06`

In [47]:
timestamp = "2016-06-28"
timestamp_lags = ["2016-05-28", "2016-04-28", "2016-03-28", "2016-02-28", "2016-01-28",
                  "2015-12-28", "2015-11-28", "2015-10-28", "2015-09-28", "2015-08-28",
                 "2015-07-28", "2015-06-28"]

X_2016_06 = extract_X_test(df_train, df_test, timestamp, timestamp_lags)
X_2016_06.shape

2016-05-28 _LAG1 (929615, 57)
2016-04-28 _LAG2 (925252, 57)
2016-03-28 _LAG3 (920975, 57)
2016-02-28 _LAG4 (915679, 57)
2016-01-28 _LAG5 (909885, 57)
2015-12-28 _LAG6 (903429, 57)
2015-11-28 _LAG7 (896458, 57)
2015-10-28 _LAG8 (881573, 57)
2015-09-28 _LAG9 (854574, 57)
2015-08-28 _LAG10 (832230, 57)
2015-07-28 _LAG11 (818424, 57)
2015-06-28 _LAG12 (622404, 57)


(929615, 690)

In [48]:
X_2016_06.to_csv(os.path.join(OUT_DIR2, "X_2016_06.csv"), index=False)

# Use `2015-06`  with 4-month lags to predict `2016-06`

## Extract `X_2015_05`, `y_2015_05`

In [ ]:
timestamp = "2015-05-28"
timestamp_lags = ["2015-04-28", "2015-03-28", "2015-02-28", "2015-01-28"]

X_2015_05, y_2015_05 = extract_X_y_train(df_train, timestamp, timestamp_lags)

In [ ]:
X_2015_05.to_csv(os.path.join(OUT_DIR3, "X_2015_05.csv"), index=False)
y_2015_05.to_csv(os.path.join(OUT_DIR3, "y_2015_05.csv"), index=False)

## Extract `X_2015_06`, `y_2015_06`

In [ ]:
timestamp = "2015-06-28"
timestamp_lags = ["2015-05-28", "2015-04-28", "2015-03-28", "2015-02-28"]

X_2015_06, y_2015_06 = extract_X_y_train(df_train, timestamp, timestamp_lags)

In [ ]:
X_2015_06.to_csv(os.path.join(OUT_DIR3, "X_2015_06.csv"), index=False)
y_2015_06.to_csv(os.path.join(OUT_DIR3, "y_2015_06.csv"), index=False)

## Extract `X_2016_05`, `y_2016_05`

In [ ]:
timestamp = "2016-05-28"
timestamp_lags = ["2016-04-28", "2016-03-28", "2016-02-28", "2016-01-28"]

X_2016_05, y_2016_05 = extract_X_y_train(df_train, timestamp, timestamp_lags)

In [ ]:
X_2016_05.to_csv(os.path.join(OUT_DIR3, "X_2016_05.csv"), index=False)
y_2016_05.to_csv(os.path.join(OUT_DIR3, "y_2016_05.csv"), index=False)

## Extract `X_2016_06`

In [ ]:
timestamp = "2016-06-28"
timestamp_lags = ["2016-05-28", "2016-04-28", "2016-03-28", "2016-02-28"]

X_2016_06 = extract_X_test(df_train, df_test, timestamp, timestamp_lags)

In [ ]:
X_2016_06.to_csv(os.path.join(OUT_DIR3, "X_2016_06.csv"), index=False)